Note, still in the process of experimenting with parameters

Import packages

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import datetime as dt
import talos as ta
from talos.model.normalizers import lr_normalizer

In [151]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

In [34]:
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 8
config.inter_op_parallelism_threads = 8
tf.session(config=config)

AttributeError: module 'tensorflow' has no attribute 'session'

Use prepared dataset

In [107]:
data = pds.read_pickle('dataset_model2v2_w_label.pkl')
dataset = data.iloc[0:10000]
labels = dataset['label']
dataset = dataset.drop(columns=['label'])

dataset.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,dir_sin,dir_cos,speed,ttjs,latitude,longitude,journey_number
0,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326317,0.924521,1
1,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326237,0.925287,1
2,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326156,0.925670,1
3,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326076,0.926437,1
4,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326116,0.926437,1


Select an arbitrary number of test journeys

In [109]:
# Roughly 20%
num_test_segments = 3
msk = np.random.randint(1, dataset.journey_number.unique()[-1], num_test_segments)
mask = dataset['journey_number'].isin(msk)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [110]:
train_input = dataset[~mask]
test_input = dataset[mask]

train_labels = labels[~mask]
test_labels = labels[mask]

tr_dat = dataset.drop(columns=['journey_number'])

In [111]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey_number'])
test_data = test_input.drop(columns=['journey_number'])

Prepare model for parameter optimization

In [112]:
keras.backend.clear_session()

In [ ]:
def ogt_model(tr_d, tr_l, val_d, val_l, params):

    model = keras.Sequential([
        keras.layers.Dense(params['layer1'], 
                           activation=params['activation'], 
                           input_shape=(tr_d.shape[1])),
        keras.layers.Dense(params['layer2']),
        keras.layers.Dropout(params['dropout1']),
        keras.layers.Dense(params['layer3']),
        keras.layers.Dropout(params['dropout2']),
        keras.layers.Dense(params['layer4']),
        keras.layers.Dense(1, activation=params['last_activation'])
    ])
    
    model.compile(loss='mae', 
                  optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])),
                  metrics=['mae'])

    # Initial_epoch can be set to resume training after preciously run epochs
    out = model.fit(tr_d.values, 
                    tr_l.values, 
                    epochs = params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=[val_d, val_l])
   
    return out, model

Define parameters to optimize over

In [139]:
p = {'lr': (2, 10, 30),
     'layer1':[19, 32, 64],
     'layer2':[19, 32, 64],
     'layer3':[19, 32, 64],
     'layer4':[19, 32, 64],
     'batch_size': [2, 5, 10],
     'epochs': [10],
     'dropout1': (0, 0.40, 10),
     'dropout2': (0, 0.40, 10),
     'optimizer': [keras.optimizers.Adam, keras.optimizers.Nadam, keras.optimizers.Adadelta],
     #'losses': [categorical_crossentropy, logcosh],
     'activation':[tf.nn.sigmoid, tf.nn.relu, tf.nn.elu],
     'last_activation': [tf.nn.softmax, tf.nn.relu]}

pt = {'lr': [2, 0.04],
     'layer1':[1, 2, 3 ,4 ,5, 6, 7],
     'layer2':[1],
     'batch_size': [10, 30, 100],
     'epochs': [1],
     'optimizer': [keras.optimizers.Adadelta],
     #'losses': [categorical_crossentropy, logcosh],
     'activation':['relu'],#tf.nn.softmax],
     'last_activation': ['softmax']}#tf.nn.relu]}

Run optimization

In [170]:
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adadelta

def make_model(optimizer = Adadelta, lr = 0.01, l1 = 19, l2 = 19, act1 = 'relu', act2='sigmoid'):
    
    model = Sequential()
    model.add(Dense(l1, activation=act1))
    model.add(Dense(l2, activation=act2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mae', 
                  optimizer=optimizer,
                  metrics=['mae'])

    return model

In [171]:
model = KerasRegressor(build_fn=make_model)

In [172]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_data.values, train_labels.values)



exception calling callback for <Future at 0x7ff352a6ab70 state=finished raised BrokenProcessPool>
sklearn.externals.joblib.externals.loky.process_executor._RemoteTraceback: 
'''
Traceback (most recent call last):
  File "/home/jacke/plugg/tdde19/tf-playground/venv/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py", line 393, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'make_model' on <module 'sklearn.externals.joblib.externals.loky.backend.popen_loky_posix' from '/home/jacke/plugg/tdde19/tf-playground/venv/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/popen_loky_posix.py'>
'''

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/jacke/plugg/tdde19/tf-playground/venv/lib/pyt

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [168]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

NameError: name 'grid_result' is not defined

In [140]:
def simple_model(tr_d, tr_l, val_d, val_l, params):
    model = keras.Sequential([
        keras.layers.Dense(params['layer1'], 
                           activation=params['activation'], 
                           input_shape=(tr_d.shape[1],)),
        #keras.layers.Dense(params['layer2']),
        keras.layers.Dense(1, activation=params['last_activation'])
    ])

    model.compile(loss='mae', 
                  optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])),
                  metrics=['mae', 'accuracy'])

    model.fit(tr_d,
              tr_l,
              epochs = params['epochs'],
              batch_size=params['batch_size'],
              validation_data=[val_d, val_l])
 
    return model

h = ta.Scan(train_data.values, train_labels.values,
            params=pt,
            dataset_name='talos_testing',
            experiment_no='1',
            val_split=0,
            x_val = test_data.values,
            y_val = test_labels.values,
            model=simple_model,
            reduction_metric=keras.metrics.mae,
            clear_tf_session=False
           )

















  0%|          | 0/42 [00:00<?, ?it/s]

Train on 7202 samples, validate on 2798 samples
Epoch 1/1
7202/7202 [==============================] - 2s 239us/step - loss: 52.5904 - mean_absolute_error: 52.5904 - acc: 0.0125 - val_loss: 53.0340 - val_mean_absolute_error: 53.0340 - val_acc: 0.0118


















  2%|▏         | 1/42 [00:02<01:33,  2.28s/it]

Train on 7202 samples, validate on 2798 samples
Epoch 1/1
7202/7202 [==============================] - 1s 111us/step - loss: 52.5904 - mean_absolute_error: 52.5904 - acc: 0.0125 - val_loss: 53.0340 - val_mean_absolute_error: 53.0340 - val_acc: 0.0118


















  5%|▍         | 2/42 [00:03<01:20,  2.01s/it]

...

Train on 7202 samples, validate on 2798 samples
Epoch 1/1
7202/7202 [==============================] - 2s 231us/step - loss: 52.5904 - mean_absolute_error: 52.5904 - acc: 0.0125 - val_loss: 53.0340 - val_mean_absolute_error: 53.0340 - val_acc: 0.0118


















 98%|█████████▊| 41/42 [01:41<00:02,  2.95s/it]

Train on 7202 samples, validate on 2798 samples
Epoch 1/1
7202/7202 [==============================] - 2s 234us/step - loss: 52.5904 - mean_absolute_error: 52.5904 - acc: 0.0125 - val_loss: 53.0340 - val_mean_absolute_error: 53.0340 - val_acc: 0.0118


















100%|██████████| 42/42 [01:44<00:00,  2.95s/it]

















Scan Finished!
